# Enregistrer les DF en SQL / SQLite

In [1]:
import pandas as pd
import numpy as np
#import request
import os
import sqlalchemy
from dotenv import load_dotenv
from projetressourcesagricoles.params import DATA_FOLDER

load_dotenv()
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PUBLIC_IP_ADDRESS = os.getenv("DB_PUBLIC_IP_ADDRESS")
DB_NAME = os.getenv("DB_NAME")
DB_CNX = f"mysql+pymysql://root:{DB_PASSWORD}@{DB_PUBLIC_IP_ADDRESS}/{DB_NAME}"


def df_optimized(df, verbose=True, **kwargs):
    """
    Reduces size of dataframe by downcasting numeircal columns
    :param df: input dataframe
    :param verbose: print size reduction if set to True
    :param kwargs:
    :return: df optimized
    """
    in_size = df.memory_usage(index=True).sum()
    # Optimized size here
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
            if type == "float":
                df[col] = pd.to_numeric(df[col], downcast="integer")
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100
    GB = out_size / 1000000000
    if verbose:
        print("optimized size by {} % | {} GB".format(ratio, GB))
    return df

## Télécharge les données et référentiels du serveur FAO

In [4]:
lang = 'fr'
URLS_DATASET = [
    {
        'file_path' : f'Commerce_MatricesCommerceDetaillees_F_Toutes_les_Données.csv',
    },
    {
        'file_path' : f'https://fenixservices.fao.org/faostat/api/v1/{lang}/definitions/domain/TM/item?output_type=objects',
    },
    {
        'file_path' : f'https://fenixservices.fao.org/faostat/api/v1/{lang}/definitions/domain/TM/reporterarea?output_type=objects',
    },
    {
        'file_path' : f'https://fenixservices.fao.org/faostat/api/v1/{lang}/definitions/domain/TM/element?output_type=objects',
    }
]

## TODO télécharger les fichiers dans un rep temporaire et trouver comment s'authentifier sur le serveur FAO

## Préprocess DB
- Chargement du DF
- Suppression des colonnes de label dont nous avons le code dans un référentiel
- Optimisation de la taille des numerics 

In [3]:
CSV_FILES_EXC = [
    {
        'file_path' : 'https://fenixservices.fao.org/faostat/api/v1/fr/definitions/domain/TM/item?output_type=objects',
        'file_path' : os.path.join(DATA_FOLDER, 'Commerce_MatricesCommerceDetaillees_F_Toutes_les_Données.csv'),
        'remove_cols': ['Pays déclarants', 'Pays partenaires', 'Produit', 'Élément'],
        'table_name': 'exchanges',
        'optimize_numerics': True,
        'read_options' : {'encoding' : 'iso-8859-1'}
    },
    # {
    #     'file_path' : 'https://fenixservices.fao.org/faostat/api/v1/fr/definitions/domain/TM/item?output_type=objects',
    #     'file_path' : os.path.join(DATA_FOLDER, 'Codes_produits.csv'),
    #     'remove_cols': [],
    #     'table_name': 'items',
    #     'optimize_numerics': False,
    #     'read_options' : {}
    # },
    # {
    #     'file_path' : 'https://fenixservices.fao.org/faostat/api/v1/fr/definitions/domain/TM/reporterarea?output_type=objects',
    #     'file_path' : os.path.join(DATA_FOLDER, 'Codes_pays.csv'),
    #     'remove_cols': [],
    #     'table_name': 'areas',
    #     'optimize_numerics': False,
    #     'read_options' : {}
    # },
    # {
    #     'file_path' : 'https://fenixservices.fao.org/faostat/api/v1/fr/definitions/domain/TM/element?output_type=objects',
    #     'file_path' : os.path.join(DATA_FOLDER, 'Codes_elements.csv'),
    #     'remove_cols': [],
    #     'table_name': 'elements',
    #     'optimize_numerics': False,
    #     'read_options' : {}
    # }
]

CSV_FILES_PROD = [
    {
        'file_path' : os.path.join(DATA_FOLDER, 'Production_Crops_Livestock_E_All_Data_(Normalized).csv'),
        'remove_cols': ['Area', 'Item', 'Year Code', 'Element'],
        'table_name': 'productions',
        'optimize_numerics': True,
        'read_options' : {'encoding': 'iso-8859-1'}
    },
    {
        'file_path' : os.path.join(DATA_FOLDER, 'Production_Crops_Livestock_E_Flags.csv'),
        'remove_cols': [],
        'table_name': 'productions_flags',
        'optimize_numerics': True,
        'read_options' : {'encoding': 'iso-8859-1'}
    },
    {
        'file_path' : 'https://fenixservices.fao.org/faostat/api/v1/fr/definitions/domain/TM/item?output_type=objects',
        'file_path' : os.path.join(DATA_FOLDER, 'Codes_produits.csv'),
        'remove_cols': [],
        'table_name': 'items',
        'optimize_numerics': False,
        'read_options' : {}
    },
    {
        'file_path' : 'https://fenixservices.fao.org/faostat/api/v1/fr/definitions/domain/TM/reporterarea?output_type=objects',
        'file_path' : os.path.join(DATA_FOLDER, 'Codes_pays.csv'),
        'remove_cols': [],
        'table_name': 'areas',
        'optimize_numerics': False,
        'read_options' : {}
    },
    {
        'file_path' : 'https://fenixservices.fao.org/faostat/api/v1/fr/definitions/domain/TM/element?output_type=objects',
        'file_path' : os.path.join(DATA_FOLDER, 'Codes_elements.csv'),
        'remove_cols': [],
        'table_name': 'elements',
        'optimize_numerics': False,
        'read_options' : {},
    }
]

CSV_FILES = CSV_FILES_EXC

In [4]:
TO_PROD_SERVER = False

for data in CSV_FILES:
    # file_path, remove_cols, table_name, optimize_numerics, read_options = data.values()
    # print (file_path, remove_cols, table_name)

    try:
        encoding = 'utf-8'
        if 'encoding' in data['read_options']:
            encoding = data['read_options']['encoding']
        tmp_df = pd.read_csv(data['file_path'], encoding=encoding, low_memory=False)
    except Exception as e:
        print(e)
        break

    if data['remove_cols']:
        tmp_df.drop(columns=data['remove_cols'], inplace=True)

    if data['optimize_numerics']:
        tmp_df = df_optimized(tmp_df)

    if TO_PROD_SERVER:
        engine = sqlalchemy.create_engine(DB_CNX)
    else:
        engine = sqlalchemy.create_engine('sqlite:///exch.db', echo=False)

    with engine.begin() as connection:
        tmp_df.to_sql(
            data['table_name'],
            con=connection,
            if_exists='replace',
            chunksize=100_000,
            index=False
        )





optimized size by 27.0 % | 2.443491932 GB


In [3]:
engine = sqlalchemy.create_engine(DB_CNX)
        
with engine.begin() as connection:
    tmp_df.to_sql(
        'exchanges',
        con=connection,
        if_exists='replace',
        chunksize=100_000,
        index=False
    )
